In [1]:
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm

import gym
import time
import flexible_bus
import json
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import os

import pandas as pd
import torch
import torch.nn as nn

In [ ]:
def traj_simulate(seed,gamma,base_policy):
    
    """
    Func to Simulate the FRD process
        return: trajectory, return list
    
    """
    # np.random.seed(seed)
    trajectory = {
        "observations": [],
        "direct_actions":[],
        "actions": [],
        "return": 0.0
    }

    env = gym.make('FlexibleBus-v0')
    obs = env.reset()
    r = 0
    done = False
    
    while not done:
        trajectory["observations"].append(obs.tolist() if isinstance(obs, np.ndarray) else obs)
        obs = torch.tensor(obs,dtype=torch.float32)
        [p1, p2] = base_policy(obs)
        p1 = p1.item()
        p2 = p2.item()
        direct_action = [p1, p2]
        trajectory["direct_actions"].append(direct_action.tolist() if isinstance(direct_action, np.ndarray) else direct_action)
        deviate_1 = int(np.random.choice([0, 1], p=[1-p1, p1]))
        deviate_2 = int(np.random.choice([0, 1], p=[1-p2, p2]))
        action = [deviate_1,deviate_2]  
        trajectory["actions"].append(action.tolist() if isinstance(action, np.ndarray) else action)
        obs, rewards, done, info = env.step(action)
        r = r * gamma + rewards
    trajectory["return"] = r
    return trajectory, r

In [3]:
class Behavioural(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)

In [34]:
traj_num = 1000000
gamma = 0.99
behavior_pi = Behavioural(input_dim=5, hidden_dim=64, output_dim=2)
# 2. Load the weights into it
behavior_pi.load_state_dict(torch.load("./Behavioural_model.pth"))
trajectory, r = traj_simulate(gamma=gamma, base_policy=behavior_pi)
# Ensure the key names match the output of traj_simulate
trajectory

TypeError: traj_simulate() missing 1 required positional argument: 'seed'

In [36]:
df = pd.read_parquet("./Traject/trajectories.pq")
df

,trajectory_id,timestep,return,obs_0,obs_1,obs_2,obs_3,obs_4,action_0,action_1,direct_action_0,direct_action_1
0,0,0,17.340915,2,0,0,1,3,1,0,0.597431,0.201027
1,0,1,17.340915,1,1,1,1,1,1,1,0.597337,0.601088
2,0,2,17.340915,3,0,1,0,1,0,0,0.198423,0.203257
3,0,3,17.340915,1,0,1,0,0,0,0,0.199607,0.600914
4,0,4,17.340915,1,0,0,0,0,0,0,0.599690,0.600836
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,999,0,15.713082,3,0,0,0,1,1,0,0.599205,0.203135
4996,999,1,15.713082,2,0,0,0,0,0,1,0.600103,0.601844
4997,999,2,15.713082,0,0,1,1,0,0,0,0.198490,0.599924
4998,999,3,15.713082,1,0,4,0,0,0,1,0.197274,0.601102
